<a href="https://colab.research.google.com/github/ByungjunKim/EssentialHumanitiesData/blob/main/data_handling_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 인문 데이터 기초: 파일 다루기

이 노트북은 인문학 연구에서 자주 사용되는 다양한 형식의 파일을 다루는 기본적인 방법을 설명합니다.

## 목차
1. CSV/TSV 파일 다루기
2. 텍스트 파일 다루기
3. JSON 파일 다루기
4. XML 파일 다루기
5. 데이터 분석 및 저장

## 필요한 라이브러리 불러오기와 경로 설정

In [ ]:
import pandas as pd
import json
import xml.etree.ElementTree as ET
import matplotlib.pyplot as plt
import os

# 데이터 폴더 경로 설정
DATA_DIR = './EssentialHumanitiesData/data'

In [ ]:
!git clone https://github.com/ByungjunKim/EssentialHumanitiesData.git

## 1. CSV/TSV 파일 다루기

### 1.1 CSV 파일 읽기

In [ ]:
# CSV 파일 읽기 (등장인물 정보)
csv_path = os.path.join(DATA_DIR, 'characters.csv')
df = pd.read_csv(csv_path)
print("CSV 파일 내용 (등장인물):")
df.head()

In [ ]:
# TSV 파일 읽기 (주요 사건)
tsv_path = os.path.join(DATA_DIR, 'events.tsv')
df_tsv = pd.read_csv(tsv_path, sep='\t')
print("TSV 파일 내용 (주요 사건):")
df_tsv.head()

### 1.2 특정 데이터 선택하기

In [ ]:
# 특정 열 선택
print("등장인물의 이름과 신분:")
print(df[['이름', '신분']])

# 조건으로 선택
print("\n주요 인물 정보:")
print(df[df['이름'].isin(['복녀', '김 주사'])])

## 2. 텍스트 파일 다루기

In [ ]:
# 텍스트 파일 읽기
txt_path = os.path.join(DATA_DIR, 'novel.txt')
with open(txt_path, 'r', encoding='utf-8') as file:
    text = file.read()
print("텍스트 파일 미리보기 (처음 200자):")
print(text[:200])

In [ ]:
# 텍스트 파일의 6번째 줄부터 본문 읽기
txt_path = os.path.join(DATA_DIR, 'novel.txt')
with open(txt_path, 'r', encoding='utf-8') as file:
    lines = file.readlines() ## line별로 읽기
    text_body = ''.join(lines[5:]) ## 6번째 줄부터 본문 읽기

print("텍스트 파일 본문 미리보기 (처음 200자):")
print(text_body[:200])

In [ ]:
# 텍스트 파일의 6번째 줄부터 본문 읽기
txt_path = os.path.join(DATA_DIR, 'novel.txt')
with open(txt_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()  # line별로 읽기
    text_body = ''.join(lines[5:])  # 6번째 줄부터 본문 읽기

# 문단별로 구분
paragraphs = text_body.split('\n\n')

# 데이터프레임으로 만들기
df_paragraphs = pd.DataFrame(paragraphs, columns=['문단'])
# 문단 index 컬럼 추가
df_paragraphs['index'] = range(1, len(df_paragraphs) + 1)
# 문단 index 컬럼을 왼쪽에 추가
df_paragraphs = df_paragraphs[['index', '문단']]
print("\n문단별 데이터프레임 미리보기:")
print(df_paragraphs.head())

## 3. JSON 파일 다루기

In [ ]:
# JSON 파일 읽기
json_path = os.path.join(DATA_DIR, 'gamja.json')
with open(json_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

# 기본 정보 출력
print("작품 정보:")
print(f"제목: {data['작품']['제목']}")
print(f"저자: {data['작품']['저자']}")
print(f"출판년도: {data['작품']['출판년도']}")

# JSON을 DataFrame으로 변환
characters_data = data['작품']['주요_등장인물']
df_characters = pd.DataFrame(characters_data)
print("\n등장인물 정보:")
print(df_characters[['이름', '신분', '나이']])

## 4. XML(TEI) 파일 다루기

In [ ]:
# XML 파일 읽기
xml_path = os.path.join(DATA_DIR, 'gamja.xml')
tree = ET.parse(xml_path)
root = tree.getroot()

# 기본 정보 추출
ns = {'tei': 'http://www.tei-c.org/ns/1.0'}
title = root.find('.//tei:title', ns).text
author = root.find('.//tei:author', ns).text
print(f"제목: {title}")
print(f"작가: {author}")

# 등장인물 정보 추출
characters = []
for person in root.findall('.//tei:person', ns):
    character = {
        '이름': person.find('tei:persName', ns).text,
        '특성': [trait.text for trait in person.findall('tei:trait', ns)],
        '거주지': person.find('tei:residence', ns).text
    }
    characters.append(character)

df_xml_characters = pd.DataFrame(characters)
print("\n등장인물 정보 (XML):")
print(df_xml_characters)

## 5. 데이터 분석 및 저장

In [ ]:
# 처리된 데이터를 저장할 폴더 생성
processed_dir = os.path.join(DATA_DIR, 'processed')
if not os.path.exists(processed_dir):
    os.makedirs(processed_dir)

# Excel 파일로 저장
output_path = os.path.join(processed_dir, '감자_분석.xlsx')
with pd.ExcelWriter(output_path) as writer:
    df_characters.to_excel(writer, sheet_name='등장인물_JSON', index=False)
    df_xml_characters.to_excel(writer, sheet_name='등장인물_XML', index=False)
print(f"데이터가 '{output_path}' 파일로 저장되었습니다.")